In [3]:
from nbconvert import HTMLExporter
import codecs
import nbformat
exporter = HTMLExporter()
output_notebook = nbformat.read('file_format.ipynb', as_version=4)
output, resources = exporter.from_notebook_node(output_notebook)
codecs.open('test.html', 'w', encoding='utf-8').write(output)

# 1. Read from US 1 min

In [ ]:
import csv

fout=open("DSM/out.dat","a")
o=csv.writer(fout)
for year in range(2006,2016):
    for month in range(1,13):
        for line in open("DSM/DSM"+str(year)+"%02d"%(month)+".dat"):
            o.writerow(line.replace("[", "").replace("]", "").split())
fout.close()

df = pd.read_csv("DSM/out.dat", skipinitialspace=True, error_bad_lines=False)

df['HrMn'] = df['date'].apply(lambda x: x[11:15])
df['date'] = df['date'].apply(lambda x: x[3:11])

df.to_csv('DSM/DSM.csv', index=False)

# 2. US 1 min to Hour avg

In [ ]:
df = pd.read_csv('DSM/DSM.csv',header=0, skipinitialspace=True, dtype={'HrMn': object})

df['speed']=pd.to_numeric(df['speed'], downcast='integer', errors='coerce')
df['dir']=pd.to_numeric(df['dir'], downcast='integer', errors='coerce')
df['dir_max']=pd.to_numeric(df['dir_max'], downcast='integer', errors='coerce')
df['speed_max']=pd.to_numeric(df['speed_max'], downcast='integer', errors='coerce')
df = df.dropna()
df=df[['date','HrMn','dir','speed','dir_max','speed_max']]

In [ ]:
df['Hr']=df['HrMn'].apply(lambda x: int(str(x)[:-2]))
df.dir = np.radians(df.dir)
temp_df =df.groupby(['date', 'Hr']).agg({'speed' : np.mean,'dir': sp.stats.circmean,'speed_max': np.max}).reset_index()
df.dir = np.degrees(df.dir)
temp_df.dir = np.degrees(temp_df.dir)
temp_df['HrMn']=temp_df['Hr'].apply(lambda x: "%02d00"%x)

## Time Comparison

In [ ]:
bins=arange(0,40)
df['speed'].hist(bins=bins, normed=True,alpha=0.5,label='min')
temp_df['speed'].hist(bins=bins, normed=True,alpha=0.5,label='hour')
plt_configure(legend=True)

In [ ]:
bins=arange(0,361)
df['dir'].hist(bins=bins, normed=True,alpha=0.5,label='min')
temp_df['dir'].hist(bins=bins, normed=True,alpha=0.5,label='hour')
plt_configure(legend=True)

In [ ]:
check_time_shift(df)
check_time_shift(temp_df)

In [ ]:
temp_df.to_csv('DSM/DSM_hr_avg.csv', index=False)

# 3. Read from Chinese Data sets

In [1]:
import csv

ipath= "./data/NCDC/cn/上海/上海/dat.txt"
uipath = unicode(ipath , "utf8")

with open(uipath) as fin, open('./data/NCDC/test.txt', 'w') as fout:
    o=csv.writer(fout)
    for line in fin:
        o.writerow(line.split())